Scrape le site booking.com pour avoir le Top 20 des hôtels par ville

In [1]:
import logging
import os 
import scrapy
from scrapy.crawler import CrawlerProcess
from variables import best_cities, hotels_jl_name


# Classe qui hérite de scrapy.Spider, qui va permettre de scrapper Booking
class BookingSpider(scrapy.Spider):

    name = "booking"

    def __init__(self, id_city, city, name = None, **kwargs):
        super().__init__(name, **kwargs)
        self.id_city = id_city
        self.city = city
        
        # URL de recherche booking la plus simple, avec 3 paramètres : 
        # la ville, le type détablissement (uniquement hôtels), triés par notes utilisateurs (puisqu'on veut le top 20 des hôtels)
        self.start_urls = [
            f"https://www.booking.com/searchresults.fr.html?ss={self.city}&order=bayesian_review_score&nflt=ht_id%3D204"
        ]

    # Parse la page de résultat de la recherche pour en extraire : 
    # le nom de l'hôtel, le lien vers sa page détaillée, la note utilisateurs
    def parse(self, response):
        nb_hotel = 1
        quotes = response.xpath("//*[@id='bodyconstraint-inner']/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div")

        # Je boucle sur chaque rectangle de la page
        for quote in quotes:
            # On veut le top 20 des hôtels
            if nb_hotel <= 20:
                name_hotel = quote.xpath("div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a/div[1]/text()").get()
                link = response.urljoin(quote.xpath("div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a/@href").get())
                score = quote.xpath("div[1]/div[2]/div/div[1]/div[2]/div/div/a/span/div/div[1]/text()").get()
                        
                if name_hotel and link:
                    # On passe les informations récupérées à la fonction parse_hotel qui va scraper la page détaillée de chaque hôtel
                    yield response.follow(link, callback=self.parse_hotel, meta={"id_city": self.id_city, "city": self.city, "name": name_hotel, "link": link, "score": score})
                    nb_hotel += 1
            else:
                break
    
    # Parse la page détaillée de chaque hôtel
    def parse_hotel(self, response):
        # On récupère les informations passées en paramètre
        id_city = response.meta["id_city"]
        city = response.meta["city"]
        name = response.meta["name"]
        link = response.meta["link"]
        score = response.meta["score"]

        description = response.xpath("//*[@id='basiclayout']/div[1]/div[2]/div/div[1]/div[1]/div[1]/div/div/p[1]/text()").get()
        coords = latitude = response.xpath("//*[@id='map_trigger_header_pin']").attrib["data-atlas-latlng"]
        
        if description and coords:
            description = description.replace("\n", " ")
            latitude = coords.split(",")[0]
            longitude = coords.split(",")[1]

            # On retroune les informations 
            yield {
                "id_city": id_city,
                "city": city,
                "hotel_name": name,
                "hotel_link": link,
                "hotel_score": score,
                "hotel_description": description,
                "hotel_lat": latitude,
                "hotel_lon": longitude
            }

# On stocke les résultats du scrapping dans un fichier .jl (json lines), chaque ligne est un objet json
# Supprimer le fichier s"il existe déjà dans le répertoire courant
if os.path.exists(hotels_jl_name):
    os.remove(hotels_jl_name)

process = CrawlerProcess(settings = {
    "USER_AGENT": "Chrome/97.0",
    "LOG_LEVEL": logging.INFO,
    # jsonlines car je lance plusieurs crawler en même temps, ici chaque élément va être écrit sur une ligne indépendante
    "FEEDS": {hotels_jl_name : {"format": "jsonlines", "encoding": "utf-8"}}
})

# Scrappe Booking pour chaque ville
for i in range(len(best_cities)): 
    process.crawl(BookingSpider, id_city = i + 1, city = best_cities[i])

process.start()

2025-04-27 16:53:22 [scrapy.utils.log] INFO: Scrapy 2.12.0 started (bot: scrapybot)
2025-04-27 16:53:22 [scrapy.utils.log] INFO: Versions: lxml 5.3.1.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.10.0, w3lib 2.1.2, Twisted 23.10.0, Python 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.5.0 8 Apr 2025), cryptography 43.0.0, Platform Windows-11-10.0.26100-SP0
2025-04-27 16:53:22 [scrapy.addons] INFO: Enabled addons:
[]
2025-04-27 16:53:22 [scrapy.extensions.telnet] INFO: Telnet Password: 89966b3cfd1fa850
2025-04-27 16:53:22 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2025-04-27 16:53:22 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2025-04-27 16:53:22 [scrapy.middleware] INFO: Enabled downloader middle